In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
data = requests.get('https://api.covid19india.org/state_district_wise.json').json()
df_statecode = pd.DataFrame.from_dict(data)
df_statecode = df_statecode.drop(['districtData']).T
state_to_statecode_dict = dict(zip(df_statecode.index, df_statecode['statecode']))
statecode_to_state_dict = dict(zip(df_statecode['statecode'], df_statecode.index))

In [ ]:
data = requests.get('https://api.covid19india.org/v4/data-all.json').json()

for date in data.keys():
    date_dict = data[date]
    # Remove all the states which don't have district data in them
    date_dict = {state : state_dict for state, state_dict in date_dict.items() if 'districts' in state_dict.keys()}
    data[date] = date_dict
    
# Remove all the dates which have 0 states with district data after pruning
data = {date : date_dict for date, date_dict in data.items() if len(date_dict) > 0}

# Make the districts key data the only data available for the state key
for date in data.keys():
    for state in data[date].keys():
        # Make the districts key dict the main dict itself for a particular date, state
        data[date][state] = data[date][state]['total']
        
    date_dict = {state: state_dict for state, state_dict in data[date].items() \
                 if {'confirmed', 'recovered', 'deceased'} <= state_dict.keys()}
    data[date] = date_dict
    
# Remove all the dates which have 0 states with district data after pruning
data = {date : date_dict for date, date_dict in data.items() if len(date_dict) > 0}

df_states_all = pd.DataFrame(columns=['date', 'state', 'confirmed', 'active', 'recovered', 'deceased', 'tested', 'migrated'])
for date in data.keys():
    df_date = pd.DataFrame.from_dict(data[date]).T.reset_index()
    df_date = df_date.rename({'index' : 'state'}, axis='columns')
    df_date['active'] = df_date['confirmed'] - (df_date['recovered'] + df_date['deceased'])
    df_date['state'] = pd.Series([statecode_to_state_dict[state_code] for state_code in df_date['state']])
    df_date['date'] = date
    df_states_all = pd.concat([df_states_all, df_date], ignore_index=True)

In [ ]:
df_states_all

In [ ]:
df_mumbai = df_districts_all.loc[(df_districts_all['state'] == 'Maharashtra') & (df_districts_all['district'] == 'Mumbai'), :]

In [ ]:
numeric_cols = ['confirmed', 'active', 'recovered', 'deceased', 'tested', 'migrated']
df_mumbai.loc[:, numeric_cols] = df_mumbai.loc[:, numeric_cols].apply(pd.to_numeric)

In [ ]:
df_mumbai